## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("..\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ancud,CL,-41.8697,-73.8203,54.54,80,13,3.83,few clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,40.66,87,75,35.68,drizzle
2,2,Sao Filipe,CV,14.8961,-24.4956,72.90,78,8,11.39,clear sky
3,3,Geraldton,AU,-28.7667,114.6000,59.41,94,40,8.05,light rain
4,4,Hermanus,ZA,-34.4187,19.2345,61.81,74,57,5.70,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp_user=float( input("Set the maximum temperature: "))
min_temp_user=float( input("Set the minimum temperature: "))

Set the maximum temperature: 90
Set the minimum temperature: 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities=city_data_df.loc[ (city_data_df['Max Temp']<=max_temp_user) & 
                                  (city_data_df['Max Temp']>=min_temp_user)  ]
preferred_cities.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,78,75,8.05,broken clouds
10,10,Hirara,JP,24.8000,125.2833,77.52,85,94,4.74,overcast clouds
21,21,Hilo,US,19.7297,-155.0900,77.29,94,100,6.91,moderate rain
23,23,Meulaboh,ID,4.1363,96.1285,78.62,86,100,3.85,moderate rain
24,24,Nanakuli,US,21.3906,-158.1547,75.15,71,54,9.89,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

Unnamed: 0             0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities.dropna()
clean_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Arraial Do Cabo,BR,75.16,broken clouds,-22.9661,-42.0278,
10,Hirara,JP,77.52,overcast clouds,24.8000,125.2833,
21,Hilo,US,77.29,moderate rain,19.7297,-155.0900,
23,Meulaboh,ID,78.62,moderate rain,4.1363,96.1285,
24,Nanakuli,US,75.15,broken clouds,21.3906,-158.1547,
27,Boueni,YT,82.38,few clouds,-12.9025,45.0761,
28,Rikitea,PF,76.53,few clouds,-23.1203,-134.9692,
29,Labuhan,ID,81.16,overcast clouds,-6.8844,112.2051,
34,Kuito,AO,75.34,scattered clouds,-12.3833,16.9333,
35,Praya,CV,79.34,few clouds,14.9215,-23.5087,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
parameters = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
             }

# 6b. Iterate through the hotel DataFrame.
for i,row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng'] 
    parameters['location']=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url_base="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(url_base,params=parameters).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[i,'Hotel Name']=hotels['results'][0]['name']
    except IndexError:
        print("Hotel not found...skipping. ")
        

Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 
Hotel not found...skipping. 


In [9]:
# 7. Drop the rows where there is no Hotel Name.
for i,row in hotel_df.iterrows():
    if (len(row['Hotel Name']))==0:
        hotel_df=hotel_df.drop([i])

clean_hotel_df=hotel_df

In [10]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Current Description} and {Max Temp} °F</dd>

</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30,31),zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))